# Replicating Research from Spring

by Joshua Delgadillo

 # Update 5

### Working on:
 1. How to Jupyter Notebooks
 2. Relevant Githubs
 3. New (but similiar??) data
 4. The Alternative Algorithm

### Relevant GitHubs

Analysis using k-mer composition
https://anderson-github-classroom.github.io/csc-448-project/eagranof/

Even more analysis using k-mer composition
https://anderson-github-classroom.github.io/csc-448-project/skurdogh/ 

Vitulgin Experimentation
https://anderson-github-classroom.github.io/csc-448-project/cilg/

The Levenshtein distance experiment
https://anderson-github-classroom.github.io/csc-448-project/awengel/ 

Mutation Rate Comparison and Spike Proteins
https://anderson-github-classroom.github.io/csc-448-project/pamidi/

### New Sample Data

The [NCBI Nucleotide Database] was used to get data last time: "This database provides multiple SARS-Cov-2 sequences, but in an effort to focus our analysis, we select an arbitrarily random subset of sequences to analyze". I have a simple thought: we choose another arbitrarily random subset of sequences to analyze, which will put us at arbitrary<sup>2</sup> (aka where you want to be)

According to the [report from Spring], 677 sequences were used. Therefore, we should use 678 to literally 1-up them. We can compare our dataset to theirs to ensure everything looks right.

[NCBI Nucleotide Database]: https://www.ncbi.nlm.nih.gov/nuccore
[report from Spring]: https://nbviewer.jupyter.org/github/anderson-github-classroom/csc-448-project/blob/master/students/enewcome/supplemental-table/table.ipynb


### The Alternative Algorithm

Needleman-Wunsch Algorithm or the Smith-Waterman Algorithm can be used to compare the similiarity of two viruses. The former was used in the Spring, so we will use the Smith-Waterman Algorithm.

From what I can tell (and I'm not smart...so grains of salt), NWA fins the optimal global alignment and SWA find the best local alignment, which means: "something". I need to look back at my sequence alignment notes, but here are my current thoughts: If we run the SWA algorithm and the local alignment is enormous, then the strands of RNA are very similiar.

Some pseduo code:

~~~

Given: String s1 with length m , String s2 with length n

    // initialize matrix, M
    
    // score cells in matrix
    for i=1 to m
        for j=1 to n
        
            // initialization: max is 0
            max = 0 
            
            // first comparison: west cell (deletion)
            score = M[i][j-1] + gapScore
            if( score > max )
                max = score
            
            // second comparison: north cell (insertion)
            score = M[i-1][j] + gapScore
            if( score > max )
                max = score
            
            // last comparison: north-west cell (alignment)
            base1 = s1[j-1]
            base2 = s2[i-1]
            
            if( base1 == base2 )              // match
                alignmentScore = matchScore
            else                              // mismatch
                alignmentScore = mismatchScore
            
            score = M[i-1][j-1] + alignmentScore
            if( score > max )
                max = score
            
            // finished all comparisons
            M[i][j] = max
    
    // return completed matrix
    return M
~~~

### Stray Thoughts

1. There were doubts about Virulign, can't we just use Blast?
2. Need to verify my beliefs with bio.
3. Shoudl we use only the most recent data.
4. SWA is hopefully mostly done after our lab.

 # Update 6

### Feedback

Professor Anderson asked us to consider if we should or shouldn't implement the algorithm by hand. The primary benefit would be our familiarization with the code and control over parameters, but we give up precious time. 

We decided to try using BLAST after all, but <b>is it actually what we need?</b> Names can be very misleading.

### Proving we can use BLAST

Proof by obvious: 
~~~

lemma 1: Smith-Waterman is a "local aligment" algorithm.

B.L.A.S.T. stands for basic "local alignment" search tool.

Done.
~~~

From the [BLAST website]: "The Basic Local Alignment Search Tool (BLAST) finds regions of local similarity between sequences. The program compares nucleotide or protein sequences to sequence databases and calculates the statistical significance of matches."

I found out BLAST actually uses a faster version of Smith-Waterman. A heuristic is used to eliminate sequences that are not likely to be a good local alignment. The shortcut was necessary because the algorithm is too slow when ran with large datasets. source: [The NCBI Handbook]

[BLAST website]: https://blast.ncbi.nlm.nih.gov/Blast.cgi
[The NCBI Handbook]: https://www.unmc.edu/bsbc/docs/NCBI_blast.pdf

### Sample Data

I pulled (at the time) recent data in the [NCBI Virus] database. There are strands of SARS-CoV-1, SARS-CoV-2, and MERS-CoV. I unironically-ironically tried 678 sequences because 677 were in the Spring. I had just read about how the alogrithm struggles with large datasets  (and crossing 678 sequences with 678 sequences is enormous. We decided to scale it back a lot becuase logic supersedes sarcasm. Instead, the CS team is each going to run blast on one of each sequence and then we'll cross reference our results to ensure we all did it correctly. From there, we can run BLAST on as many sequences as we want. The function "print_fasta" does exactly what you think it does: prints a .fasta file. 

[NCBI Virus]: https://www.ncbi.nlm.nih.gov/labs/virus/vssi/#/

In [51]:
from Bio import SeqIO
import pandas as pd 
  
def print_fasta(virus_name, file_name):
    # initialize list of lists 
    data = []

    fasta_sequences = SeqIO.parse(open(file_name),'fasta')
    with open(file_name) as out_file:
        for fasta in fasta_sequences:
            name, sequence = fasta.id, str(fasta.seq)
            data.append([name, sequence])

    # Create the pandas DataFrame 
    df = pd.DataFrame(data, columns = [virus_name, 'sequence']) 

    # print dataframe. 
    print(df) 
        

Now we can print out our three sequences.

In [57]:
print_fasta("SARS-CoV-1", "SARS-CoV-1.fasta")
print()
print_fasta("SARS-CoV-2", "SARS-CoV-2.fasta")
print()
print_fasta("MERS-CoV", "MERS-CoV.fasta")
print()

   SARS-CoV-1                                           sequence
0  AGT21091.1  MESLVLGVNEKTHVQLSLPVLQVRDVLVRGFGDSVEEALSEAREHL...

   SARS-CoV-2                                           sequence
0  QOP83741.1  MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHL...

     MERS-CoV                                           sequence
0  QLD98029.1  MSFVAGVIAQGARGTYRAALNSEKHQDHVSLTVPLCGSGNLVEKLS...



### BLASTing Seqeunces

First, we need to create a blast database for our three datasets.

In [58]:
%%bash
/usr/local/ncbi-blast-2.10.1+/bin/makeblastdb -in SARS-CoV-1.fasta -dbtype prot
/usr/local/ncbi-blast-2.10.1+/bin/makeblastdb -in SARS-CoV-2.fasta -dbtype prot
/usr/local/ncbi-blast-2.10.1+/bin/makeblastdb -in MERS-CoV.fasta -dbtype prot
cp *.fasta ./data/
mv *.fasta.* ./data/



Building a new DB, current time: 10/26/2020 21:48:22
New DB name:   /home/jupyter-jdelga26/448/SARS-CoV-1.fasta
New DB title:  SARS-CoV-1.fasta
Sequence type: Protein
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 1 sequences in 0.000182152 seconds.




Building a new DB, current time: 10/26/2020 21:48:22
New DB name:   /home/jupyter-jdelga26/448/SARS-CoV-2.fasta
New DB title:  SARS-CoV-2.fasta
Sequence type: Protein
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 1 sequences in 0.000196934 seconds.




Building a new DB, current time: 10/26/2020 21:48:22
New DB name:   /home/jupyter-jdelga26/448/MERS-CoV.fasta
New DB title:  MERS-CoV.fasta
Sequence type: Protein
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 1 sequences in 0.000197887 seconds.




We did it! Where are our diplomas? 

Next, we run blast on every pair (the number of threads was bumped up to the max becuase why not?):

In [60]:
%%bash
/usr/local/ncbi-blast-2.10.1+/bin/blastp -query ./data/SARS-CoV-1.fasta -db ./data/SARS-CoV-2.fasta -evalue 1e-6 -num_threads 16 -out ./data/blast_S1_S2.txt
/usr/local/ncbi-blast-2.10.1+/bin/blastp -query ./data/SARS-CoV-1.fasta -db ./data/MERS-CoV.fasta -evalue 1e-6 -num_threads 16 -out ./data/blast_S1_M.txt
/usr/local/ncbi-blast-2.10.1+/bin/blastp -query ./data/SARS-CoV-2.fasta -db ./data/MERS-CoV.fasta -evalue 1e-6 -num_threads 16 -out ./data/blast_S2_M.txt

Print out BLAST result of SARS-CoV-1 and SARS-CoV-2

In [61]:
!cat ./data/blast_S1_S2.txt

BLASTP 2.10.1+


Reference: Stephen F. Altschul, Thomas L. Madden, Alejandro A.
Schaffer, Jinghui Zhang, Zheng Zhang, Webb Miller, and David J.
Lipman (1997), "Gapped BLAST and PSI-BLAST: a new generation of
protein database search programs", Nucleic Acids Res. 25:3389-3402.


Reference for composition-based statistics: Alejandro A. Schaffer,
L. Aravind, Thomas L. Madden, Sergei Shavirin, John L. Spouge, Yuri
I. Wolf, Eugene V. Koonin, and Stephen F. Altschul (2001),
"Improving the accuracy of PSI-BLAST protein database searches with
composition-based statistics and other refinements", Nucleic Acids
Res. 29:2994-3005.



Database: SARS-CoV-2.fasta
           1 sequences; 7,096 total letters



Query= AGT21091.1 |replicase polyprotein 1a [SARS coronavirus wtic-MB]

Length=4382
                                                                      Score     E
Sequences producing significant alignments:                          (Bits)  Value

QOP83741.1 |ORF1ab polyprotein [Severe acute re

Print out BLAST result of SARS-CoV-1 and MERS

In [62]:
!cat ./data/blast_S1_M.txt

BLASTP 2.10.1+


Reference: Stephen F. Altschul, Thomas L. Madden, Alejandro A.
Schaffer, Jinghui Zhang, Zheng Zhang, Webb Miller, and David J.
Lipman (1997), "Gapped BLAST and PSI-BLAST: a new generation of
protein database search programs", Nucleic Acids Res. 25:3389-3402.


Reference for composition-based statistics: Alejandro A. Schaffer,
L. Aravind, Thomas L. Madden, Sergei Shavirin, John L. Spouge, Yuri
I. Wolf, Eugene V. Koonin, and Stephen F. Altschul (2001),
"Improving the accuracy of PSI-BLAST protein database searches with
composition-based statistics and other refinements", Nucleic Acids
Res. 29:2994-3005.



Database: MERS-CoV.fasta
           1 sequences; 7,078 total letters



Query= AGT21091.1 |replicase polyprotein 1a [SARS coronavirus wtic-MB]

Length=4382
                                                                      Score     E
Sequences producing significant alignments:                          (Bits)  Value

QLD98029.1 |ORF1ab [Middle East respiratory syndr

Print out BLAST result of SARS-CoV-1 and SARS-CoV-2

In [63]:
!cat ./data/blast_S2_M.txt

BLASTP 2.10.1+


Reference: Stephen F. Altschul, Thomas L. Madden, Alejandro A.
Schaffer, Jinghui Zhang, Zheng Zhang, Webb Miller, and David J.
Lipman (1997), "Gapped BLAST and PSI-BLAST: a new generation of
protein database search programs", Nucleic Acids Res. 25:3389-3402.


Reference for composition-based statistics: Alejandro A. Schaffer,
L. Aravind, Thomas L. Madden, Sergei Shavirin, John L. Spouge, Yuri
I. Wolf, Eugene V. Koonin, and Stephen F. Altschul (2001),
"Improving the accuracy of PSI-BLAST protein database searches with
composition-based statistics and other refinements", Nucleic Acids
Res. 29:2994-3005.



Database: MERS-CoV.fasta
           1 sequences; 7,078 total letters



Query= QOP83741.1 |ORF1ab polyprotein [Severe acute respiratory syndrome
coronavirus 2]

Length=7096
                                                                      Score     E
Sequences producing significant alignments:                          (Bits)  Value

QLD98029.1 |ORF1ab [Middle East 